# Assignment 1

You only need to write one line of code for each question. When answering questions that ask you to identify or interpret something, the length of your response doesn’t matter. For example, if the answer is just ‘yes,’ ‘no,’ or a number, you can just give that answer without adding anything else.

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that **no outside searches are required by the assignment!**). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

### Classification using KNN

Let's set up our workspace and use the **Wine dataset** from `scikit-learn`. This dataset contains 178 wine samples with 13 chemical features, used to classify wines into different classes based on their origin.

The **response variable** is `class`, which indicates the type of wine. We'll use all of the chemical features to predict this response variable.

In [1]:
# Import standard libraries
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
from sklearn.datasets import load_wine

# Load the Wine dataset
wine_data = load_wine()

# Convert to DataFrame
wine_df = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)

# Bind the 'class' (wine target) to the DataFrame
wine_df['class'] = wine_data.target

# Display the DataFrame
wine_df


alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0      14.23        1.71  2.43               15.6      127.0           2.80   
1      13.20        1.78  2.14               11.2      100.0           2.65   
2      13.16        2.36  2.67               18.6      101.0           2.80   
3      14.37        1.95  2.50               16.8      113.0           3.85   
4      13.24        2.59  2.87               21.0      118.0           2.80   
..       ...         ...   ...                ...        ...            ...   
173    13.71        5.65  2.45               20.5       95.0           1.68   
174    13.40        3.91  2.48               23.0      102.0           1.80   
175    13.27        4.28  2.26               20.0      120.0           1.59   
176    13.17        2.59  2.37               20.0      120.0           1.65   
177    14.13        4.10  2.74               24.5       96.0           2.05   

     flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
0          3.06                  0.28             2.29             5.64  1.04   
1          2.76                  0.26             1.28             4.38  1.05   
2          3.24                  0.30             2.81             5.68  1.03   
3          3.49                  0.24             2.18             7.80  0.86   
4          2.69                  0.39             1.82             4.32  1.04   
..          ...                   ...              ...              ...   ...   
173        0.61                  0.52             1.06             7.70  0.64   
174        0.75                  0.43             1.41             7.30  0.70   
175        0.69                  0.43             1.35            10.20  0.59   
176        0.68                  0.53             1.46             9.30  0.60   
177        0.76                  0.56             1.35             9.20  0.61   

     od280/od315_of_diluted_wines  proline  class  
0                            3.92   1065.0      0  
1                            3.40   1050.0      0  
2                            3.17   1185.0      0  
3                            3.45   1480.0      0  
4                            2.93    735.0      0  
..                            ...      ...    ...  
173                          1.74    740.0      2  
174                          1.56    750.0      2  
175                          1.56    835.0      2  
176                          1.62    840.0      2  
177                          1.60    560.0      2  

[178 rows x 14 columns]

#### **Question 1:** 
#### Data inspection

Before fitting any model, it is essential to understand our data. **Use Python code** to answer the following questions about the **Wine dataset**:

_(i)_ How many observations (rows) does the dataset contain?

In [3]:
# Your answer here
print(wine_df.shape[0])

178


_(ii)_ How many variables (columns) does the dataset contain?

In [4]:
# Your answer here
print(wine_df.shape[1])

14


_(iii)_ What is the 'variable type' of the response variable `class` (e.g., 'integer', 'category', etc.)? What are the 'levels' (unique values) of the variable?

In [10]:
# Your answer here
print((wine_df["class"]).dtypes)

int32



_(iv)_ How many predictor variables do we have (Hint: all variables other than `class`)? 

In [11]:
# Your answer here
print(len(wine_df.columns) - 1)

13


You can use `print()` and `describe()` to help answer these questions.

#### **Question 2:** 
#### Standardization and data-splitting

Next, we must preform 'pre-processing' or 'data munging', to prepare our data for classification/prediction. For KNN, there are three essential steps. A first essential step is to 'standardize' the predictor variables. We can achieve this using the scaler method, provided as follows:

In [12]:
# Select predictors (excluding the last column)
predictors = wine_df.iloc[:, :-1]

# Standardize the predictors
scaler = StandardScaler()
predictors_standardized = pd.DataFrame(scaler.fit_transform(predictors), columns=predictors.columns)

# Display the head of the standardized predictors
print(predictors_standardized.head())

    alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
0  1.518613   -0.562250  0.232053          -1.169593   1.913905   
1  0.246290   -0.499413 -0.827996          -2.490847   0.018145   
2  0.196879    0.021231  1.109334          -0.268738   0.088358   
3  1.691550   -0.346811  0.487926          -0.809251   0.930918   
4  0.295700    0.227694  1.840403           0.451946   1.281985   

   total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0       0.808997    1.034819             -0.659563         1.224884   
1       0.568648    0.733629             -0.820719        -0.544721   
2       0.808997    1.215533             -0.498407         2.135968   
3       2.491446    1.466525             -0.981875         1.032155   
4       0.808997    0.663351              0.226796         0.401404   

   color_intensity       hue  od280/od315_of_diluted_wines   proline  
0         0.251717  0.362177                      1.847920  1.013009  
1        -0.293321  0.406051

(i) Why is it important to standardize the predictor variables?

 <span style="color:Green">Answer</span>:</p>Standardizing predictor variables is crucial for KNN because the algorithm relies on **distance metrics**, such as Euclidean distance, to determine the "closeness" of data points. If the predictor variables are on different scales, features with larger ranges will dominate the distance calculations, leading to **biased results**. Standardization ensures that **all features contribute equally** by scaling them to have a mean of 0 and a standard deviation of 1

(ii) Why did we elect not to standard our response variable `Class`?

<span style="color:Green">Answer</span>:</p>Since they are **not involved in distance calculations**. Keeping them in their original form ensures that their meaning is preserved and interpretable so that predictions can be interpreted correctly.

(iii) A second essential step is to set a random seed. Do so below (Hint: use the random.seed function). Why is setting a seed important? Is the particular seed value important? Why or why not?

<span style="color:Green">Answer</span>:</p>Setting a random seed ensures **reproducibility** of results. Many machine learning tasks involve randomness, such as splitting datasets into training and testing sets. A fixed seed guarantees that anyone running the code will obtain the same results, which is essential for debugging, collaboration, and validating experiments.
The specific value of the seed is generally not important; any fixed value will work for reproducibility. However, different seed values might lead to slight variations in model performance due to differences in how data is split or initialized. Here is how we can set a random seed: 

``` 
import random
import numpy as np
np.random.seed(42) 
```

(iv) A third essential step is to split our standardized data into separate training and testing sets. We will split into 75% training and 25% testing. The provided code randomly partitions our data, and creates linked training sets for the predictors and response variables. 

Extend the code to create a non-overlapping test set for the predictors and response variables.

In [54]:
# set a seed for reproducibility
np.random.seed(123)

# split the data into a training and testing set. hint: use train_test_split !
response = wine_df["class"]

X_train, X_test, y_train, y_test = train_test_split(predictors_standardized, response, test_size = 0.25, stratify = response)
X_test
#print(f"The train dataset size is: {X_train.shape}")
#print(f"The test dataset size is: {X_test.shape}")
#print(f"The train response variable size is: {y_train.shape}")
#print(f"The test response variable size is: {y_test.shape}")


alcohol  malic_acid       ash  alcalinity_of_ash  magnesium  \
102 -0.816038    0.102021  0.341713           0.451946  -0.122282   
84  -1.433671   -1.298334  0.780354          -0.448909  -0.403135   
96  -1.470729   -0.194208  1.365208           0.602088   2.405399   
65  -0.778980   -1.011081  0.707247          -0.418881  -0.122282   
79  -0.371343    1.376703  0.122392           1.052516   0.088358   
17   1.024507   -0.687923  0.926567           0.151661   1.071345   
109 -1.717782   -0.885409  1.218995           0.151661  -0.403135   
113 -1.964835   -1.432983  0.487926           0.451946  -0.824415   
28   1.073917   -0.391694  1.584529          -0.028510   0.509638   
159  0.592164   -0.598156  0.999674           0.902373  -0.754202   
38   0.085705   -0.750759 -0.974210          -1.199622  -0.122282   
34   0.629222   -0.481460  1.036228          -0.148624   0.720278   
125 -1.149560   -0.158301 -0.718336           0.451946  -1.035055   
115 -2.434235   -0.741782 -0.608676           0.602088  -1.035055   
71   1.061565   -0.741782  1.109334           1.653086  -0.964842   
76   0.036295   -1.289357 -2.399793          -1.049479  -0.964842   
131 -0.148995    0.586759  0.122392           0.151661   0.298998   
33   0.938038   -0.723829  1.218995           0.001518   2.264972   
60  -0.828391   -1.109824 -0.316249          -1.049479   0.088358   
19   0.789806    0.685502  0.707247          -1.289707   1.141558   
114 -1.137207   -0.849502  0.487926           0.902373  -1.105268   
47   1.110975   -0.589180 -0.901103          -1.049479   0.088358   
48   1.358028   -0.283974  0.122392          -0.208681   0.228785   
158  1.654492   -0.589180  1.218995           1.653086  -0.122282   
133 -0.371343    1.089450 -0.023821           0.602088   0.439425   
137 -0.581338    2.848870  0.999674           1.653086  -0.262708   
154 -0.519575   -0.939268 -0.974210           0.151661   0.228785   
136 -0.927212    2.139716  0.634140           0.451946  -0.754202   
2    0.196879    0.021231  1.109334          -0.268738   0.088358   
168  0.715690    0.218717  1.182441           1.502943   0.369212   
117 -0.717217   -0.652016 -0.645229           0.902373   0.579852   
32   0.839217   -0.454530 -0.023821          -0.689137   0.298998   
22   0.876275   -0.427600 -0.023821          -0.869308   0.088358   
108 -0.964270   -0.939268 -1.559065          -0.148624  -0.543562   
73  -0.013116   -0.598156  0.853460           3.154511   2.756465   
77  -1.433671    0.496993 -0.499016          -0.448909   0.860705   
142  0.641574    0.748338  1.292101           1.202658  -0.192495   
9    1.061565   -0.885409 -0.352802          -1.049479  -0.122282   
85  -0.408401   -1.217544 -0.462462          -0.448909  -0.052068   
58   0.888627   -0.813595  0.487926          -0.839280   0.579852   
45   1.493907    1.529305  0.268606          -0.178653   0.790492   
175  0.332758    1.744744 -0.389355           0.151661   1.422412   
42   1.086270   -0.400670  0.816907          -1.349764   0.088358   
143  0.765101    2.346179 -0.060375           0.151661  -0.543562   
177  1.395086    1.583165  1.365208           1.502943  -0.262708   

     total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
102       0.424438    0.081051             -0.176095        -0.492158   
84       -0.152402    0.181447             -1.143031         1.330009   
96       -1.113800   -1.043392             -1.787656        -0.054137   
65        0.200111    0.623193              0.065639         0.856946   
79        0.857067    0.522796              0.549108         0.629175   
17        1.049347    1.376168              0.307374         0.226196   
109       0.712858    0.894264             -0.578985         1.575301   
113       0.296251   -0.019346              0.468530        -0.264388   
28        1.049347    0.944462              0.065639         0.296279   
159       0.488531   -0.932956              1.274310         1.224884   
38        0.168065    0.6131

#### **Question 3:**
#### Model initialization and cross-validation
We are finally set to fit the KNN model. 


Perform a grid search to tune the `n_neighbors` hyperparameter using 10-fold cross-validation. Follow these steps:

1. Initialize the KNN classifier using `KNeighborsClassifier()`.
2. Define a parameter grid for `n_neighbors` ranging from 1 to 50.
3. Implement a grid search using `GridSearchCV` with 10-fold cross-validation to find the optimal number of neighbors.
4. After fitting the model on the training data, identify and return the best value for `n_neighbors` based on the grid search results.

In [47]:
# Initialize the KNN classifier
knn = KNeighborsClassifier()

# Define a parameter grid for n_neighbors ranging from 1 to 50
parameter_grid = {'n_neighbors': list(range(1, 51))}

# Perform a grid search using GridSearchCV with 10-fold cross-validation
wine_tune_grid = GridSearchCV(
    estimator = knn, 
    param_grid = parameter_grid, 
    cv=10
    )

# Fitting the model
wine_tune_grid.fit(X_train, y_train)

# Identify and report the best value for n_neighbors
best_n_neighbors = wine_tune_grid.best_params_
print(f"Best value for n_neighbors is: {best_n_neighbors} ")




Best value for n_neighbors is: {'n_neighbors': 7} 


#### **Question 4:**
#### Model evaluation

Using the best value for `n_neighbors`, fit a KNN model on the training data and evaluate its performance on the test set using `accuracy_score`.

In [53]:
# Fit the KNN model on the training data
knn_model = KNeighborsClassifier(**wine_tune_grid.best_params_)
knn_model.fit(X_train, y_train)

# Predict on the test set
y_pred = knn_model.predict(X_test)

# Evaluate performance using accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {100 * accuracy:.2f}%")

Accuracy on the test set: 93.33%


# Criteria


| **Criteria**                                           | **Complete**                                      | **Incomplete**                                    |
|--------------------------------------------------------|---------------------------------------------------|--------------------------------------------------|
| **Data Inspection**                                    | Data is inspected for number of variables, observations and data types. | Data inspection is missing or incomplete.         |
| **Data Scaling**                                       | Data scaling or normalization is applied where necessary (e.g., using `StandardScaler`). | Data scaling or normalization is missing or incorrectly applied. |
| **Model Initialization**                               | The KNN model is correctly initialized and a random seed is set for reproducibility.            | The KNN model is not initialized, is incorrect, or lacks a random seed for reproducibility. |
| **Parameter Grid for `n_neighbors`**                   | The parameter grid for `n_neighbors` is correctly defined. | The parameter grid is missing or incorrectly defined. |
| **Cross-Validation Setup**                             | Cross-validation is set up correctly with 10 folds. | Cross-validation is missing or incorrectly set up. |
| **Best Hyperparameter (`n_neighbors`) Selection**       | The best value for `n_neighbors` is identified using the grid search results. | The best `n_neighbors` is not selected or incorrect. |
| **Model Evaluation on Test Data**                      | The model is evaluated on the test data using accuracy. | The model evaluation is missing or uses the wrong metric. |


## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applying_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-4-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
